# This is a sample Jupyter Notebook

Below is an example of a code cell. 
Put your cursor into the cell and press Shift+Enter to execute it and select the next one, or click 'Run Cell' button.

Press Double Shift to search everywhere for classes, files, tool windows, actions, and settings.

To learn more about Jupyter Notebooks in PyCharm, see [help](https://www.jetbrains.com/help/pycharm/ipython-notebook-support.html).
For an overview of PyCharm, go to Help -> Learn IDE features or refer to [our documentation](https://www.jetbrains.com/help/pycharm/getting-started.html).

In [52]:
# Load All Necessary Imports
from statsmodels.regression.rolling import RollingOLS
import datetime as dt
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
import statsmodels.api as sm
import warnings
import numpy as np
import pandas_ta
import pandas_datareader.data as web
warnings.filterwarnings('ignore')

ImportError: cannot import name 'NaN' from 'numpy' (C:\Users\N\miniconda3\Lib\site-packages\numpy\__init__.py)

If Dataframe is in the data folder, gotta read from file in next lines. Since I'm lazy, I am using online data.

In [12]:
list_of_SP500_companies_wikipedia = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
sp500 = pd.read_html(list_of_SP500_companies_wikipedia)
# 2 Dfs returned of which we are interested in the first
sp500 = sp500[0]

,Symbol,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded
0,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1957-03-04,66740,1902
1,AOS,A. O. Smith,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,91142,1916
2,ABT,Abbott Laboratories,Health Care,Health Care Equipment,"North Chicago, Illinois",1957-03-04,1800,1888
3,ABBV,AbbVie,Health Care,Biotechnology,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
4,ACN,Accenture,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373,1989
...,...,...,...,...,...,...,...,...
498,XYL,Xylem Inc.,Industrials,Industrial Machinery & Supplies & Components,"White Plains, New York",2011-11-01,1524472,2011
499,YUM,Yum! Brands,Consumer Discretionary,Restaurants,"Louisville, Kentucky",1997-10-06,1041061,1997
500,ZBRA,Zebra Technologies,Information Technology,Electronic Equipment & Instruments,"Lincolnshire, Illinois",2019-12-23,877212,1969
501,ZBH,Zimmer Biomet,Health Care,Health Care Equipment,"Warsaw, Indiana",2001-08-07,1136869,1927


In [14]:
# Cleaning up the data
sp500['Symbol'] = sp500['Symbol'].str.replace('.','-')

# Check if changes came into effect
symbols_list = sp500['Symbol'].unique().tolist()
symbols_list

['MMM',
 'AOS',
 'ABT',
 'ABBV',
 'ACN',
 'ADBE',
 'AMD',
 'AES',
 'AFL',
 'A',
 'APD',
 'ABNB',
 'AKAM',
 'ALB',
 'ARE',
 'ALGN',
 'ALLE',
 'LNT',
 'ALL',
 'GOOGL',
 'GOOG',
 'MO',
 'AMZN',
 'AMCR',
 'AEE',
 'AEP',
 'AXP',
 'AIG',
 'AMT',
 'AWK',
 'AMP',
 'AME',
 'AMGN',
 'APH',
 'ADI',
 'ANSS',
 'AON',
 'APA',
 'APO',
 'AAPL',
 'AMAT',
 'APTV',
 'ACGL',
 'ADM',
 'ANET',
 'AJG',
 'AIZ',
 'T',
 'ATO',
 'ADSK',
 'ADP',
 'AZO',
 'AVB',
 'AVY',
 'AXON',
 'BKR',
 'BALL',
 'BAC',
 'BAX',
 'BDX',
 'BRK-B',
 'BBY',
 'TECH',
 'BIIB',
 'BLK',
 'BX',
 'BK',
 'BA',
 'BKNG',
 'BWA',
 'BSX',
 'BMY',
 'AVGO',
 'BR',
 'BRO',
 'BF-B',
 'BLDR',
 'BG',
 'BXP',
 'CHRW',
 'CDNS',
 'CZR',
 'CPT',
 'CPB',
 'COF',
 'CAH',
 'KMX',
 'CCL',
 'CARR',
 'CAT',
 'CBOE',
 'CBRE',
 'CDW',
 'CE',
 'COR',
 'CNC',
 'CNP',
 'CF',
 'CRL',
 'SCHW',
 'CHTR',
 'CVX',
 'CMG',
 'CB',
 'CHD',
 'CI',
 'CINF',
 'CTAS',
 'CSCO',
 'C',
 'CFG',
 'CLX',
 'CME',
 'CMS',
 'KO',
 'CTSH',
 'CL',
 'CMCSA',
 'CAG',
 'COP',
 'ED',
 'STZ',
 

Crucial moment. We have our tickers for the SP500 sorted, now we decide from what date the data can be counted as relevant. My argument is the market correction after the 2008 crash occured by logarithmic adjustment in August 2010 so we are approximating around then. There were no other global events except COVID that would decrease the quality of the data.

Problems:
Data might be too general
Long for the GARCH model to process
Hard to adjust for COVID year

In [33]:
end_date = dt.date.today()
start_date_15_years = pd.to_datetime(end_date) - pd.DateOffset(365*15)
start_date_10_years = pd.to_datetime(end_date) - pd.DateOffset(365*10)
start_date_5_years = pd.to_datetime(end_date) - pd.DateOffset(365*5)

In [34]:
# Downloads all the SNP500 Constituents

#15 years of data. Can be altered later to see whether predictability increases or decreases
df_15 = yf.download(symbols_list, start=start_date_15_years, end=end_date)
df_15 = df_15.stack()

[*********************100%***********************]  503 of 503 completed

1 Failed download:
['CTAS']: YFPricesMissingError('$%ticker%: possibly delisted; no price data found  (1d 2010-01-24 00:00:00 -> 2025-01-20)')


In [40]:
# Reorganize data
df_15.index.names = ['date', 'ticker']
df_15.columns = df_15.columns.str.lower()
df_15

Price               adj close       close        high         low        open  \
date       ticker                                                               
2010-01-25 A        18.832973   21.065807   21.208870   20.908442   21.044350   
           AAPL      6.117826    7.252500    7.310714    7.149643    7.232500   
           ABT      18.884943   26.245060   26.633698   26.163494   26.273848   
           ACGL      7.574436    7.965556    8.060000    7.917778    8.018889   
           ACN      31.797529   41.880001   42.540001   41.619999   42.540001   
...                       ...         ...         ...         ...         ...   
2025-01-17 XYL     119.660004  119.660004  120.830002  119.250000  119.980003   
           YUM     125.320000  125.320000  127.349998  125.190002  126.910004   
           ZBH     109.559998  109.559998  109.970001  108.470001  108.989998   
           ZBRA    405.709991  405.709991  407.290009  402.290009  406.040009   
           ZTS     165.889999  165.889999  169.669998  165.809998  168.899994   

Price                    volume  
date       ticker                
2010-01-25 A       3.608518e+06  
           AAPL    1.065700e+09  
           ABT     1.010942e+07  
           ACGL    3.288600e+06  
           ACN     3.665700e+06  
...                         ...  
2025-01-17 XYL     1.396500e+06  
           YUM     1.604900e+06  
           ZBH     1.097100e+06  
           ZBRA    2.706000e+05  
           ZTS     4.656700e+06  

[1797512 rows x 6 columns]

Technical Indicators used for each stock: (Can be appended and changed later)
Garman-Klass Volatility
RSI
Bollinger Bands
ATR
MACD
Dollar Volume

In [41]:
# Insert Garman Class Volatility Column
df_15['garman_klass_vol'] = ((np.log(df_15['high']) - np.log(df_15['low'])) **2) / 2 - ( 2 * np.log(2) - 1) * (np.log(df_15['adj close']) - np.log(df_15['open']) **2)
df_15

Price               adj close       close        high         low        open  \
date       ticker                                                               
2010-01-25 A        18.832973   21.065807   21.208870   20.908442   21.044350   
           AAPL      6.117826    7.252500    7.310714    7.149643    7.232500   
           ABT      18.884943   26.245060   26.633698   26.163494   26.273848   
           ACGL      7.574436    7.965556    8.060000    7.917778    8.018889   
           ACN      31.797529   41.880001   42.540001   41.619999   42.540001   
...                       ...         ...         ...         ...         ...   
2025-01-17 XYL     119.660004  119.660004  120.830002  119.250000  119.980003   
           YUM     125.320000  125.320000  127.349998  125.190002  126.910004   
           ZBH     109.559998  109.559998  109.970001  108.470001  108.989998   
           ZBRA    405.709991  405.709991  407.290009  402.290009  406.040009   
           ZTS     165.889999  165.889999  169.669998  165.809998  168.899994   

Price                    volume  garman_klass_vol  
date       ticker                                  
2010-01-25 A       3.608518e+06          2.451664  
           AAPL    1.065700e+09          0.812853  
           ABT     1.010942e+07          2.992090  
           ACGL    3.288600e+06          0.892155  
           ACN     3.665700e+06          4.097450  
...                         ...               ...  
2025-01-17 XYL     1.396500e+06          7.005082  
           YUM     1.604900e+06          7.196196  
           ZBH     1.097100e+06          6.687395  
           ZBRA    2.706000e+05         11.616652  
           ZTS     4.656700e+06          8.189112  

[1797512 rows x 7 columns]

In [53]:
# Insert RSI column with length = length
length = 20
